# Specifying the requirement packages with versions necessary for the model.

In [1]:
!pip install numpy
!pip install scipy
!pip install pandas
!pip install scikit-learn
!pip install tensorflow
!pip install h5py
!pip install torch
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 15.8/15.8 MB 3.0 MB/s eta 0:00:00


You should consider upgrading via the 'D:\psudo_desktop\Master\Sem 1\Google-Quick-Draw\run\Scripts\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 46.2/46.2 MB 2.7 MB/s eta 0:00:00


You should consider upgrading via the 'D:\psudo_desktop\Master\Sem 1\Google-Quick-Draw\run\Scripts\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 11.6/11.6 MB 1.8 MB/s eta 0:00:00
     -------------------------------------- 229.9/229.9 KB 3.5 MB/s eta 0:00:00
     -------------------------------------- 345.4/345.4 KB 4.3 MB/s eta 0:00:00
     -------------------------------------- 505.5/505.5 KB 4.5 MB/s eta 0:00:00


...

You should consider upgrading via the 'D:\psudo_desktop\Master\Sem 1\Google-Quick-Draw\run\Scripts\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     -------------------------------------- 159.8/159.8 MB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 1.7/1.7 MB 1.8 MB/s eta 0:00:00
     -------------------------------------- 133.3/133.3 KB 2.6 MB/s eta 0:00:00
     -------------------------------------- 228.5/228.5 MB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 2.4 MB/s eta 0:00:00
     -------------------------------------- 316.1/316.1 KB 1.6 MB/s eta 0:00:00
     -------------------------------------- 286.4/286.4 KB 2.0 MB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 2.3 MB/s eta 0:00:00
     -------------------------------------- 536.2/536.2 KB 5.6 MB/s eta 0:00:00


You should consider upgrading via the 'D:\psudo_desktop\Master\Sem 1\Google-Quick-Draw\run\Scripts\python.exe -m pip install --upgrade pip' command.


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     -------------------------------------- 132.1/132.1 KB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for pickle5
Failed to build pickle5
  Running setup.py install for pickle5: started
  Running setup.py install for pickle5: finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [36 lines of output]
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.10
  creating build\lib.win-amd64-3.10\pickle5
  copying pickle5\pickle.py -> build\lib.win-amd64-3.10\pickle5
  copying pickle5\pickletools.py -> build\lib.win-amd64-3.10\pickle5
  copying pickle5\__init__.py -> build\lib.win-amd64-3.10\pickle5
  creating build\lib.win-amd64-3.10\pickle5\test
  copying pickle5\test\pickletester.py -> build\lib.win-amd64-3.10\pickle5\test
  copying pickle5\test\test_pickle.py -> build\lib.win-amd64-3.10\pickle5\test
  copying pickle5\test\test_picklebuffer.py -> build\lib.win-amd64-3.10\pickle5\test
  copying pickle5\test\__init__.py -> build\lib.win-amd64-3.10\pickle5\test
  running build_ext
  building 'pickle5._pickle' extension
  creating build\temp.win-amd64-3.10
  creating build\temp.win-amd64-3.10\Release

In [ ]:
!pip egrep 'matplotlib|numpy|pandas|scikit-learn|tensorflow|torch'

In [ ]:
# !pip uninstall keras
!pip uninstall keras

# Imports

In [1]:
import tensorflow as tf

# import tensorflow.keras

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Conv2D, Activation, MaxPooling2D, Flatten, Dense)
from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import pickle
import numpy as np
from keras.utils.np_utils import to_categorical

# Dataset

In [3]:
# gpus = tf.config.experimental.list_physical_devices('GPU')
# print(gpus)
# tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
# tf.config.experimental.set_memory_growth(gpus[0], True)

In [4]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


## Loading data from pkl file

In [5]:
pkl = open('train100c5k_v2.pkl', 'rb')
im = pickle.load(pkl)

## Preprocessing the data and creating training testing split

In [6]:
data = im['data'].values
lable = im['target'].values

# print(lable.shape)

data = [x for x in data]
data = np.array(data)
data = data.reshape(500000,28,28,1)

label = LabelBinarizer()
lable = label.fit_transform(lable)

train_data, test_data, train_lable, test_lable = train_test_split(data, lable, test_size=0.05,random_state=420)

# scaleing the data
train_data = train_data / 255.0
test_data = test_data / 255.0


print("data shape",train_data.shape,test_data.shape)
print("lable shape",train_lable.shape,test_lable.shape)


data shape (475000, 28, 28, 1) (25000, 28, 28, 1)
lable shape (475000, 100) (25000, 100)


# Classifier Model

In [7]:
class Im_classfification:

    def construct(width, height, depth, classes):
        model = Sequential()
        inputShape = (width, height, depth)

        if K.image_data_format() == "channels_first":
            inputShape = (depth, width, height)
            print(inputShape)

        # first set of convolution net
        # print(inputShape,"asdasdsad")
        model.add(Conv2D(32, (5,5), strides=(1, 1),padding="valid", input_shape=inputShape))
        model.add(layers.BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # second set of convolution net
        model.add(Conv2D(50, (5, 5), strides=(1, 1), padding="same"))
        model.add(layers.BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # third layer of convolution net
        model.add(Conv2D(64, (5, 5), strides=(1, 1), padding="valid"))
        model.add(layers.BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # Fully connect layer
        model.add(Flatten())
        model.add(Dense(512))
        model.add(layers.BatchNormalization())
        model.add(Activation('relu'))
        model.add(layers.Dropout(0.25))

        # reduction the output
        model.add(Dense(256))
        model.add(layers.BatchNormalization())
        model.add(Activation('relu'))
        model.add(layers.Dropout(0.25))

        # softmax classifier
        model.add(Dense(classes))
        model.add(Activation('softmax'))

        return model


## Compiling the model

In [8]:
print("compiling model .......")
model = Im_classfification.construct(28,28,1,100)
# model = classifier.construct(28,28,1,100)
model.compile(optimizer='adam', loss="categorical_crossentropy",
            metrics=["accuracy"])

compiling model .......


# Training the model

In [ ]:
# print("training the network .......")
H = model.fit(train_data, train_lable, batch_size=512, epochs=10, verbose=1, validation_data=(test_data,test_lable))

Epoch 1/10


## Saving the Model for further utilization

In [ ]:
print("saving the model ........")
model.save("image_recogniation.h5")

saving the model ........


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model("image_recognition.h5")

# Print the model summary
model.summary()

In [ ]:
# image_model = DenseNet121(weights='/content/drive/MyDrive/CheXNet_weights.h5', classes = 14, input_shape=(256,256,3))
# model = Model(image_model.input, image_model.layers[-2].output)
tf.keras.utils.plot_model(model, to_file='model_1_plot.png', show_shapes=True, show_layer_names=True)

## Model Evaluation

In [ ]:
predictions = model.predict(test_data, batch_size=180)
print(classification_report(test_lable.argmax(axis=1), predictions.argmax(axis=1)))

139/139 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

           0       0.63      0.83      0.72       249
           1       0.85      0.88      0.87       258
           2       0.70      0.68      0.69       252
           3       0.81      0.84      0.83       264
           4       0.80      0.75      0.77       248
           5       0.70      0.75      0.73       233
           6       0.76      0.70      0.73       264
           7       0.49      0.33      0.40       238
           8       0.76      0.79      0.78       245
           9       0.42      0.43      0.43       230
          10       0.58      0.59      0.58       252
          11       0.57      0.59      0.58       254
          12       0.71      0.78      0.74       246
          13       0.83      0.81      0.82       239
          14       0.87      0.89      0.88       230
          15       0.90      0.87      0.89       248
          16       0.80   

In [ ]:
!python /content/mini_project.py '/content/drive/MyDrive/Copy of train100c5k_v2.pkl' '/content/test100c5k_nolabel.pkl'

2022-12-05 04:31:00.332201: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-05 04:31:06.078179: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-05 04:31:06.079082: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-12-05 04:31:06.079123: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

## Github Quick draw: https://github.com/googlecreativelab/quickdraw-dataset